In [ ]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 8.7 MB/s eta 0:00:00


In [ ]:
!pip install -q transformers torchaudio pydub pyngrok gtts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip install groq flask pyngrok requests flask-cors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 6.7 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoProcessor, BarkModel
from scipy.io.wavfile import write
import numpy as np
from IPython.display import Audio, FileLink, HTML, display
import os
import json
import requests
from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading
import time
import boto3
from botocore.config import Config
from datetime import datetime

In [ ]:
# Step 1: Set up device and data type
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if "cuda" in device else torch.float32
print(f"Using device: {device}")

# Step 2: Load model and explicitly move it to the device
model = BarkModel.from_pretrained("suno/bark", torch_dtype=torch_dtype).to(device)
processor = AutoProcessor.from_pretrained("suno/bark")
print("Bark model loaded successfully!")

Using device: cuda:0


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.49G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

...

speaker_embeddings_path.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Bark model loaded successfully!


In [ ]:
from groq import Groq

# Initialize Groq client (you'll need to set your API key)
GROQ_API_KEY = ""  # Replace with your actual API key
groq_client = Groq(api_key=GROQ_API_KEY)

def detect_emotions_groq(text):
    """Detect emotions in text using Groq LLM"""
    try:
        prompt = f"""
        Analyze the emotions in the following text and return a JSON response with emotion scores (0-1):

        Text: "{text}"

        Return only a JSON object with these emotions and their confidence scores:
        {{
            "joy": 0.0,
            "sadness": 0.0,
            "anger": 0.0,
            "fear": 0.0,
            "surprise": 0.0,
            "disgust": 0.0,
            "neutral": 0.0,
            "dominant_emotion": "emotion_name"
        }}
        """

        response = groq_client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="llama3-8b-8192",
            temperature=0.3,
            max_tokens=200
        )

        result = response.choices[0].message.content.strip()
        # Extract JSON from response
        start_idx = result.find('{')
        end_idx = result.rfind('}') + 1
        json_str = result[start_idx:end_idx]

        return json.loads(json_str)
    except Exception as e:
        print(f"Error detecting emotions: {e}")
        return {
            "joy": 0.0, "sadness": 0.0, "anger": 0.0, "fear": 0.0,
            "surprise": 0.0, "disgust": 0.0, "neutral": 1.0,
            "dominant_emotion": "neutral"
        }

In [ ]:
# Supabase S3 Configuration
SUPABASE_ACCESS_KEY = "0065f886443287967b45f2ac060befbb"
SUPABASE_SECRET_KEY = "f687b0cc7273c4b2463ca0b02386e93d55272ddd3fb05925253c0521ddcc32db"
SUPABASE_ENDPOINT = "https://gbdgtveuufrfqbywgxvy.supabase.co/storage/v1/s3"
SUPABASE_REGION = "ap-south-1"
BUCKET_NAME = "audios"  # Replace with your actual bucket name

# Initialize S3 client for Supabase
s3_client = boto3.client(
    's3',
    endpoint_url=SUPABASE_ENDPOINT,
    aws_access_key_id=SUPABASE_ACCESS_KEY,
    aws_secret_access_key=SUPABASE_SECRET_KEY,
    region_name=SUPABASE_REGION,
    config=Config(signature_version='s3v4')
)

In [ ]:
def generate_speech_bark(text, emotion="neutral", output_filename="output.wav"):
    """Generate speech using Bark model with emotion consideration"""
    try:
        # Map emotions to voice presets and add emotional context
        emotion_voice_mapping = {
            "joy": "v2/en_speaker_6",
            "sadness": "v2/en_speaker_7",
            "anger": "v2/en_speaker_8",
            "fear": "v2/en_speaker_9",
            "surprise": "v2/en_speaker_6",
            "neutral": "v2/en_speaker_6"
        }

        # Add emotional context to text
        emotion_prefixes = {
            "joy": "[laugh] ",
            "sadness": "[sigh] ",
            "anger": "[frustrated] ",
            "fear": "[whisper] ",
            "surprise": "[gasp] ",
            "neutral": ""
        }

        voice_preset = emotion_voice_mapping.get(emotion, "v2/en_speaker_6")
        emotional_text = emotion_prefixes.get(emotion, "") + text

        # Process and generate
        inputs = processor(emotional_text, voice_preset=voice_preset, return_tensors="pt")
        inputs = {key: value.to(device) for key, value in inputs.items()}

        pad_token_id = processor.tokenizer.pad_token_id
        with torch.no_grad():
            audio_array = model.generate(**inputs, pad_token_id=pad_token_id).squeeze().cpu().numpy()

        # Save audio
        sample_rate = model.generation_config.sample_rate
        audio_array = audio_array.astype(np.float32)
        write(output_filename, rate=sample_rate, data=audio_array)

        return output_filename
    except Exception as e:
        print(f"Error generating speech: {e}")
        return None

In [ ]:
def upload_to_supabase(file_path, filename):
    """Upload audio file to Supabase storage and return public URL"""
    try:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        unique_filename = f"audio_{timestamp}_{filename}"

        # Upload using S3 endpoint
        s3_client.upload_file(
            file_path,
            BUCKET_NAME,
            unique_filename,
            ExtraArgs={'ContentType': 'audio/wav'}
        )

        # Generate public URL using REST API endpoint (NOT S3 endpoint)
        public_url = f"https://gbdgtveuufrfqbywgxvy.supabase.co/storage/v1/object/public/{BUCKET_NAME}/{unique_filename}"

        if os.path.exists(file_path):
            os.remove(file_path)

        return public_url

    except Exception as e:
        print(f"Error uploading to Supabase: {e}")
        return None

In [ ]:
from flask_cors import CORS  # Add this import

app = Flask(__name__)
CORS(app)

# app = Flask(__name__)

@app.route('/')
def index():
    return HTML_INTERFACE

@app.route('/detect_emotion', methods=['POST'])
def detect_emotion_endpoint():
    data = request.json
    text = data.get('text', '')

    if not text:
        return jsonify({'error': 'No text provided'}), 400

    emotions = detect_emotions_groq(text)
    return jsonify(emotions)

@app.route('/generate_speech', methods=['POST'])
def generate_speech_endpoint():
    data = request.json
    text = data.get('text', '')
    emotion = data.get('emotion', 'neutral')

    if not text:
        return jsonify({'error': 'No text provided'}), 400

    filename = f"speech_{int(time.time())}.wav"
    print(f"Generating speech for: {text}")

    local_file = generate_speech_bark(text, emotion, filename)
    print(f"Local file generated: {local_file}")

    if local_file:
        print("Attempting to upload to Supabase...")
        public_url = upload_to_supabase(local_file, filename)
        print(f"Upload result: {public_url}")

        if public_url:
            return jsonify({
                'success': True,
                'filename': filename,
                'audio_url': public_url,
                'message': 'Speech generated and uploaded successfully'
            })
        else:
            return jsonify({'error': 'Failed to upload audio to storage'}), 500
    else:
        return jsonify({'error': 'Failed to generate speech'}), 500

def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False)

In [ ]:
# Start Flask server in a separate thread
import threading
import time


def start_server():
    threading.Thread(target=run_flask, daemon=True).start()
    time.sleep(3)  # Wait for server to start

    # Create ngrok tunnel
    ngrok.set_auth_token(os.getenv("NGROK_AUTH_TOKEN", "30XiacceR6V2FrnLqzyWgJRp51x_2r5S7wUevhrWUqK7dYFLW"))
    public_url = ngrok.connect(5000)
    print(f"Flask server is running!")
    print(f"Public URL: {public_url}"
    print(f"Access your interface at: {public_url}")

    return public_url

# Don't forget to set your Groq API key first!
GROQ_API_KEY = "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"

public_url = start_server()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Flask server is running!
Public URL: NgrokTunnel: "https://9674bea1438e.ngrok-free.app" -> "http://localhost:5000"
Access your interface at: NgrokTunnel: "https://9674bea1438e.ngrok-free.app" -> "http://localhost:5000"
